In [1]:
import nltk
import math
import string
from collections import Counter
import os
import glob
from string import digits
from math import log
from pandas import DataFrame
import pandas as pd
import numpy
import re
from textblob import TextBlob

In [2]:
#from nltk.corpus import stopwords
from nltk.stem import *
from nltk.stem.porter import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

/home/aayushi/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [4]:
with open('/home/aayushi/english.txt','r') as sw:
    stopwords = sw.read().splitlines()

In [5]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove numbers
    3. Remove all stopwords
    4. Returns a list of the cleaned text
    """
    
    result = [i for i in mess if not i.isdigit()]
    
    # Check characters to see if they are in punctuation
    nopunc = [char for char in result if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords]

In [6]:
def split_into_lemmas(message):
    message = str(message).lower()
    words = TextBlob(message).words
    # for each word, take its "base form" = lemma 
    return [word.lemma for word in words]

#messages.message.head().apply(split_into_lemmas)
#corpus.head().apply(split_into_lemmas)

In [7]:
def init_lists(folder):
    a_list = []
    file_list = os.listdir(folder)
    for a_file in file_list:
        f = open(folder + a_file, 'r')
        a_list.append(f.read())
    f.close()
    return a_list

In [8]:
mri_var = init_lists('/home/aayushi/Downloads/reports/mri/')
xray_var = init_lists('/home/aayushi/Downloads/reports/x-ray/')

In [9]:
all_docs = [(i, 'mri') for i in mri_var]
all_docs += [(i, 'xray') for i in xray_var]

In [10]:
print(len(all_docs))

6


In [11]:
corpus = pd.DataFrame(all_docs, columns=['contents', 'labels'])

In [12]:
#Shuffle the code
corpus = corpus.reindex(numpy.random.permutation(corpus.index))
corpus.head()

,contents,labels
1,"\nUSARad.com\n300 Barr Harbor Drive, Suite 280...",mri
5,xMRI - Second Opinions On Demand(tm)\n309 Fell...,xray
0,"\nPatient: DOE, JOHN\nExam Date:\n05/20/2010\n...",mri
4,xMRI - Second Opinions On Demand(tm)\n309 Fell...,xray
3,"\nUSARad.com\n300 Barr Harbor Drive, Suite 280...",mri


In [13]:
corpus_lemma = pd.DataFrame(all_docs, columns=['contents', 'labels'])

In [14]:
corpus.head()

,contents,labels
1,"\nUSARad.com\n300 Barr Harbor Drive, Suite 280...",mri
5,xMRI - Second Opinions On Demand(tm)\n309 Fell...,xray
0,"\nPatient: DOE, JOHN\nExam Date:\n05/20/2010\n...",mri
4,xMRI - Second Opinions On Demand(tm)\n309 Fell...,xray
3,"\nUSARad.com\n300 Barr Harbor Drive, Suite 280...",mri


In [15]:
corpus.describe()

,contents,labels
count,6,6
unique,6,2
top,"\nUSARad.com\n300 Barr Harbor Drive, Suite 280...",mri
freq,1,4


In [16]:
corpus.groupby('labels').describe()

contents                                                               
          count unique                                                top freq
labels                                                                        
mri           4      4  \nUSARad.com\n300 Barr Harbor Drive, Suite 280...    1
xray          2      2  xMRI - Second Opinions On Demand(tm)\n309 Fell...    1

In [50]:
corpus['length']=corpus['contents'].apply(len)

In [19]:
corpus.length.describe()

count       6.000000
mean     2058.500000
std      1050.971503
min      1024.000000
25%      1337.250000
50%      1966.000000
75%      2231.000000
max      3948.000000
Name: length, dtype: float64

In [21]:
corpus.head()

,contents,labels,length
1,"\nUSARad.com\n300 Barr Harbor Drive, Suite 280...",mri,1815
5,xMRI - Second Opinions On Demand(tm)\n309 Fell...,xray,1178
0,"\nPatient: DOE, JOHN\nExam Date:\n05/20/2010\n...",mri,3948
4,xMRI - Second Opinions On Demand(tm)\n309 Fell...,xray,1024
3,"\nUSARad.com\n300 Barr Harbor Drive, Suite 280...",mri,2117


In [22]:
bow_transformer = CountVectorizer(analyzer=split_into_lemmas).fit(corpus['contents'])

# Print total number of vocabulary words
print (len(bow_transformer.vocabulary_))

515


In [23]:
print(bow_transformer.vocabulary_)

{'usarad.com': 491, '300': 23, 'barr': 73, 'harbor': 221, 'drive': 167, 'suite': 453, '280': 21, 'west': 503, 'conshohocken': 119, 'pa': 355, '19428': 17, 'phone': 372, '888.886.5238': 37, 'fax': 192, '888.886.5221': 36, 'patient': 365, 'doe': 164, 'john': 259, 'exam': 182, 'date': 138, '06/05/2010': 6, 'mrn': 318, 'jd4usarad': 258, 'dob': 162, '01/01/1961': 2, 'referring': 402, 'physician': 373, 'dr': 166, 'david': 139, 'livesey': 285, '305': 24, '418-8166': 29, 'mri': 317, 'of': 343, 'the': 475, 'brain': 85, 'without': 509, 'contrast': 124, 'history': 225, 'headache': 222, 'sequence': 422, 'sagittal': 416, 'flair': 198, 'and': 54, 'coronal': 129, 't2-weighted': 466, 'image': 232, 'are': 63, 'supplemented': 457, 'by': 92, 'axial': 71, 't1': 464, 't2': 465, 'there': 477, 'is': 253, 'diffuse': 154, 'dilatation': 155, 'sulcus': 454, 'ventricle': 495, 'more': 314, 'than': 473, 'one': 346, 'would': 512, 'expect': 184, 'for': 204, 'a': 38, 'this': 480, 'age': 48, 'group': 219, 'ha': 220, 'b

In [24]:
content_4 = corpus['contents'][4]
print (content_4)

xMRI - Second Opinions On Demand(tm)
309 Fellowship Road
Suite 200
Mt. Laurel, NJ  08054


Phone: 888-733-9674
Fax: 888-886-2486
Patient: DOE, JOHN (M)
Exam Date:
12/27/2013
MRN : JD1004
DOB:
07/08/1987
Referring Physician: XMRI, SECOND OPINION
FAX:
888-886-2486




CR OF THE LEFT KNEE COMPLETE 4 OR MORE VIEWS


CLINICAL HISTORY: Knee pain. COMMENTS:
Moderate to severe tricompartmental degenerative changes are noted, demonstrated by marked osteophytosis, endplate sclerosis and significant joint space narrowing, involving the medial compartment to the greater degree.

Large suprapatellar joint effusion is noted.

There is no fracture or dislocation. There is no chondrocalcinosis. The patella is normal in position and appearance.

IMPRESSION:

1. No fracture or dislocation.
2. Moderate to severe tricompartmental degenerative changes.
3. Large suprapatellar joint effusion.
4. Consider follow up with MRI.




-Electronically Signed by:


RADIOLOGIST, ADMIN on


12/30/2013 11:13:51 AM







In [25]:
print(len(content_4))

1024


In [26]:
#vector representation of the same content_3
bow4 = bow_transformer.transform([content_4])
print(bow4)
print (bow4.shape)

  (0, 8)	1
  (0, 9)	1
  (0, 10)	3
  (0, 13)	1
  (0, 15)	1
  (0, 16)	1
  (0, 19)	1
  (0, 20)	1
  (0, 22)	1
  (0, 25)	1
  (0, 28)	2
  (0, 34)	1
  (0, 35)	2
  (0, 45)	1
  (0, 51)	1
  (0, 54)	2
  (0, 60)	1
  (0, 63)	1
  (0, 92)	2
  (0, 103)	2
  (0, 104)	1
  (0, 106)	1
  (0, 109)	1
  (0, 112)	1
  (0, 114)	1
  :	:
  (0, 361)	1
  (0, 365)	1
  (0, 372)	1
  (0, 373)	1
  (0, 377)	1
  (0, 398)	1
  (0, 402)	1
  (0, 411)	1
  (0, 418)	1
  (0, 419)	2
  (0, 424)	2
  (0, 429)	1
  (0, 430)	1
  (0, 435)	1
  (0, 453)	1
  (0, 458)	2
  (0, 475)	4
  (0, 477)	2
  (0, 485)	1
  (0, 486)	3
  (0, 488)	2
  (0, 490)	1
  (0, 498)	1
  (0, 507)	1
  (0, 514)	2
(1, 515)


In [27]:
print(bow_transformer.get_feature_names()[486])

to


In [28]:
corpus_bow = bow_transformer.transform(corpus['contents'])

In [29]:
corpus_bow

<6x515 sparse matrix of type '<class 'numpy.int64'>'
	with 925 stored elements in Compressed Sparse Row format>

In [30]:
print('Shape of Sparse Matrix: ', corpus_bow.shape)
print('Amount of Non-Zero occurences: ', corpus_bow.nnz)

Shape of Sparse Matrix:  (6, 515)
Amount of Non-Zero occurences:  925


In [31]:
sparsity = (100.0 * corpus_bow.nnz / (corpus_bow.shape[0] * corpus_bow.shape[1]))
print('sparsity: {}'.format(round(sparsity)))

sparsity: 30


In [32]:
#TF-IDF
tfidf_transformer = TfidfTransformer().fit(corpus_bow)

In [33]:
tfidf4 = tfidf_transformer.transform(bow4)
print(tfidf4)

  (0, 514)	0.157336272899
  (0, 507)	0.042585518089
  (0, 498)	0.0786681364493
  (0, 490)	0.0959350781451
  (0, 488)	0.19187015629
  (0, 486)	0.14745031396
  (0, 485)	0.0786681364493
  (0, 477)	0.113828725216
  (0, 475)	0.170342072356
  (0, 458)	0.157336272899
  (0, 453)	0.0491501046532
  (0, 435)	0.0569143626081
  (0, 430)	0.0959350781451
  (0, 429)	0.042585518089
  (0, 424)	0.157336272899
  (0, 419)	0.157336272899
  (0, 418)	0.066417046349
  (0, 411)	0.0786681364493
  (0, 402)	0.042585518089
  (0, 398)	0.042585518089
  (0, 377)	0.0959350781451
  (0, 373)	0.042585518089
  (0, 372)	0.0491501046532
  (0, 365)	0.042585518089
  (0, 361)	0.0786681364493
  :	:
  (0, 114)	0.0959350781451
  (0, 112)	0.0959350781451
  (0, 109)	0.0786681364493
  (0, 106)	0.066417046349
  (0, 104)	0.0959350781451
  (0, 103)	0.0983002093065
  (0, 92)	0.085171036178
  (0, 63)	0.042585518089
  (0, 60)	0.0959350781451
  (0, 54)	0.085171036178
  (0, 51)	0.0786681364493
  (0, 45)	0.042585518089
  (0, 35)	0.15733627289

In [34]:
print(tfidf_transformer.idf_[bow_transformer.vocabulary_['spine']])

1.33647223662


In [35]:
# Corpus transformation
corpus_tfidf = tfidf_transformer.transform(corpus_bow)
print(corpus_tfidf.shape)

(6, 515)


In [36]:
print(corpus_tfidf)

  (0, 512)	0.053860072739
  (0, 509)	0.0319529807995
  (0, 508)	0.053860072739
  (0, 507)	0.0478169017266
  (0, 506)	0.053860072739
  (0, 505)	0.107720145478
  (0, 503)	0.0372879974315
  (0, 501)	0.107720145478
  (0, 497)	0.107720145478
  (0, 495)	0.161580218217
  (0, 493)	0.107720145478
  (0, 491)	0.0372879974315
  (0, 489)	0.044166030125
  (0, 487)	0.053860072739
  (0, 486)	0.0827818644526
  (0, 484)	0.0639059615989
  (0, 483)	0.053860072739
  (0, 480)	0.111863992295
  (0, 478)	0.053860072739
  (0, 477)	0.159764903997
  (0, 475)	0.21517605777
  (0, 474)	0.08833206025
  (0, 473)	0.08833206025
  (0, 469)	0.107720145478
  (0, 466)	0.044166030125
  :	:
  (5, 63)	0.150093638234
  (5, 62)	0.069316914192
  (5, 61)	0.0422656737526
  (5, 59)	0.034658457096
  (5, 54)	0.206378752572
  (5, 53)	0.138633828384
  (5, 49)	0.0422656737526
  (5, 46)	0.0422656737526
  (5, 45)	0.0187617047793
  (5, 38)	0.0250744975492
  (5, 37)	0.0292610509824
  (5, 36)	0.0292610509824
  (5, 31)	0.103975371288
  (5, 29)

In [37]:
#Building a model

doc_train, doc_test, label_train, label_test = \
train_test_split(corpus['contents'], corpus['labels'], test_size=0.3)

In [38]:
print ('The training size is 70% of the entire dataset. Size of Training Set: ', len(doc_train))
print ('The test size is 30% of the entire dataset. Size of Test Set:         ', len(doc_test))
print ('Total Size of the dataset:                                            ', len(doc_train) + len(doc_test))

The training size is 70% of the entire dataset. Size of Training Set:  4
The test size is 30% of the entire dataset. Size of Test Set:          2
Total Size of the dataset:                                             6


In [39]:
#Creating a Data Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=split_into_lemmas)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [40]:
pipeline.fit(doc_train,label_train)

Pipeline(steps=[('bow', CountVectorizer(analyzer=<function split_into_lemmas at 0x7fb3eb069400>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), preprocess...f=False, use_idf=True)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [51]:
predictions = pipeline.predict(doc_train)

['mri' 'mri' 'mri' 'mri']


In [52]:
print('predicted:', pipeline.predict(tfidf4)[0])

predicted: mri


In [43]:
print(len(predictions))

4


In [44]:
print ('accuracy', accuracy_score(label_train,predictions))
print ('confusion matrix\n', confusion_matrix(label_train,predictions))
print ('(row=expected, col=predicted)')

accuracy 1.0
confusion matrix
 [[2 0]
 [0 2]]
(row=expected, col=predicted)


In [45]:
print (classification_report(predictions,label_train))

             precision    recall  f1-score   support

        mri       1.00      1.00      1.00         2
       xray       1.00      1.00      1.00         2

avg / total       1.00      1.00      1.00         4



In [46]:
#For Test Set
pipeline.fit(doc_test,label_test)
predictions_testset = pipeline.predict(doc_test)
print(len(predictions_testset))
print ('accuracy', accuracy_score(label_test,predictions_testset))
print ('confusion matrix\n', confusion_matrix(label_test,predictions_testset))
print ('(row=expected, col=predicted)')

2
accuracy 1.0
confusion matrix
 [[2]]
(row=expected, col=predicted)


/home/aayushi/anaconda3/lib/python3.6/site-packages/sklearn/naive_bayes.py:455: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


In [47]:
print (classification_report(predictions_testset,label_test))

             precision    recall  f1-score   support

        mri       1.00      1.00      1.00         2

avg / total       1.00      1.00      1.00         2

